<a href="https://colab.research.google.com/github/vilim-cro/SLM-graph-extractor/blob/Few-shot-learning/SLM_graph_extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade --quiet  langchain langchain-community langchain-openai langchain-experimental neo4j langchain-google-vertexai json-repair

In [68]:
import os
from google.colab import userdata
os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('HF_TOKEN')

In [69]:
from langchain_community.llms import HuggingFacePipeline, HuggingFaceEndpoint
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.graphs import Neo4jGraph
from langchain_core.runnables import RunnableSequence
from langchain_experimental.graph_transformers import LLMGraphTransformer

In [70]:
repo_id = "mistralai/Mistral-7B-Instruct-v0.3"

llm = HuggingFaceEndpoint(
    repo_id=repo_id,
    temperature=0.1,
)

#llm = HuggingFacePipeline(pipeline=pipe)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [76]:
# Define the repo ID and parameters for Hugging Face Inference
repo_id = "mistralai/Mistral-7B-Instruct-v0.3"
llm = HuggingFaceEndpoint(
    repo_id=repo_id,
    temperature=0.1,
    max_length=150  # Adjust as needed
)

# Create a few-shot prompt with examples
prompt = """
Convert the following sentences into a graph format with entities and relationships.

Example 1:
Input: Alice gave Bob a book.
Output: Entities: [Alice, Bob, book]; Relations: [(Alice, gave, Bob)]

Example 2:
Input: John sent an email to Mary.
Output: Entities: [John, Mary, email]; Relations: [(John, sent, Mary)]

Example 3:
Input: The dog chased the cat.
Output: Entities: [dog, cat]; Relations: [(dog, chased, cat)]

Example 4:
Input: Emma painted a picture for her mother.
Output: Entities: [Emma, picture, mother]; Relations: [(Emma, painted, picture), (picture, for, mother)]

Example 5:
Input: The chef cooked dinner for the guests.
Output: Entities: [chef, dinner, guests]; Relations: [(chef, cooked, dinner), (dinner, for, guests)]

Example 6:
Input: Sarah took her dog to the park.
Output: Entities: [Sarah, dog, park]; Relations: [(Sarah, took, dog), (dog, to, park)]

Example 7:
Input: The scientist discovered a new element.
Output: Entities: [scientist, element]; Relations: [(scientist, discovered, element)]

Example 8:
Input: Michael borrowed a pen from Lisa.
Output: Entities: [Michael, pen, Lisa]; Relations: [(Michael, borrowed, pen), (pen, from, Lisa)]

Example 9:
Input: Lucy found a wallet on the ground.
Output: Entities: [Lucy, wallet, ground]; Relations: [(Lucy, found, wallet), (wallet, on, ground)]

Example 10:
Input: The teacher assigned homework to the students.
Output: Entities: [teacher, homework, students]; Relations: [(teacher, assigned, homework), (homework, to, students)]

Example 11:
Input: James bought a coffee for his friend.
Output: Entities: [James, coffee, friend]; Relations: [(James, bought, coffee), (coffee, for, friend)]

Example 12:
Input: The doctor examined the patient with care.
Output: Entities: [doctor, patient]; Relations: [(doctor, examined, patient)]

Example 13:
Input: Laura wrote a letter to her grandmother.
Output: Entities: [Laura, letter, grandmother]; Relations: [(Laura, wrote, letter), (letter, to, grandmother)]

Example 14:
Input: David watched a movie with his brother.
Output: Entities: [David, movie, brother]; Relations: [(David, watched, movie), (movie, with, brother)]

Example 15:
Input: The artist sketched a portrait of a landscape.
Output: Entities: [artist, portrait, landscape]; Relations: [(artist, sketched, portrait), (portrait, of, landscape)]

Example 16:
Input: Anna traveled to Paris last year.
Output: Entities: [Anna, Paris, year]; Relations: [(Anna, traveled to, Paris), (traveled, last, year)]

Example 17:
Input: Tom gifted a ring to his sister.
Output: Entities: [Tom, ring, sister]; Relations: [(Tom, gifted, ring), (ring, to, sister)]

Example 18:
Input: The librarian organized the books on the shelves.
Output: Entities: [librarian, books, shelves]; Relations: [(librarian, organized, books), (books, on, shelves)]

Example 19:
Input: Daniel shared his notes with his classmates.
Output: Entities: [Daniel, notes, classmates]; Relations: [(Daniel, shared, notes), (notes, with, classmates)]

Example 20:
Input: Megan practiced piano at the music school.
Output: Entities: [Megan, piano, music school]; Relations: [(Megan, practiced, piano), (piano, at, music school)]

Now, for the following sentence:

Input: At the science fair, Clara presented her project about renewable energy sources.
Output:
"""

# Use the endpoint to generate the response
response = llm(prompt)

# Print the generated result
print(response)


                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful
Entities: [Clara, project, renewable energy sources, science fair]
Relations: [(Clara, presented, project), (project, about, renewable energy sources), (presented, at, science fair)]
